In [1]:
import math
import random

### Built-in functions

#### Rastrigin

In [2]:
def Rastgirin(X: list[float], A: float) -> float:
    n = len(X)
    sum = A*n
    for i in range(n):
        sum += X[i]**2 - A * math.cos(2 * math.pi * X[i])
    return sum

#### Booth

In [3]:
def Booth(x: float, y: float) -> float:
    return (x + 2 * y - 7)**2 + (2 * x + y - 5)**2

#### Lévi

In [4]:
def Lévi(x: float, y: float) -> float:
    p1 = math.sin(3 * math.pi * x)**2
    p2 = (x - 1)**2 * (1 + math.sin(3 * math.pi * y)**2)
    p3 = (y - 1)**2 * (1 + math.sin(2 * math.pi * y)**2)
    return p1 + p2 + p3

#### functions

In [6]:
def fitness_functions(function_name: str, A: float, genes: list[float]):
    match function_name:
            case "Rastgirin":
                if A == None:
                    raise ValueError("No A value given for the Rastrigin function!")
                return Rastgirin(genes, A)
            case "Booth":
                if len(genes) != 2:
                    raise ValueError("Too many genes given!")
                return Booth(genes[0], genes[1])
            case "Lévi":
                if len(genes) != 2:
                    raise ValueError("Too many genes given!")
                return Lévi(genes[0], genes[1])
            case _:
                raise ValueError(f'"{function_name}" is not one of the built-in functions that can be used. Options: "Rastgirin", "Booth", "Lévi"')    

In [7]:
def randfloat(start: float = 0, end: float = 1, n: int = 1):
    rangelen = end - start
    if n == 1:
        return random.random() * rangelen + start
    else:
        X = []
        for i in range(n):
            X.append(random.random() * rangelen + start)
        return X

### Genetic Algorithm functions

In [68]:
class Chromosome:
    def __init__(self):
        self.genes = []
        self.fitness = None
        self.function_value = None
        self.probability = None
        self.diversity = None
        self.rank = None
    
    #------- operator operloads -------#
    def __eq__(self, other):
        if self.genes != other.genes:
            return False
        if self.fitness != other.fitness:
            return False
        if self.function_value != other.function_value:
            return False
        if self.probability != other.probability:
            return False
        if self.diversity != other.diversity:
            return False
        return True
    
    def __ne__(self, other):
        return not self == other
    
    def __lt__(self, other):
        return self.fitness < other.fitness
    def __gt__(self, other):
        return self.fitness > other.fitness
    def __le__(self, other):
        return self.fitness <= other.fitness
    def __ge__(self, other):
        return self.fitness >= other.fitness
    

    #---- other ----#
    def calculate_fitness(self, fitness_function: str, A: float = None):
        # fitness_function = "Rastgirin", "Booth", "Lévi"
        self.function_value = fitness_functions(fitness_function, A, self.genes)
        self.fitness = 1/self.function_value

    def calculate_probabilities(chromosomes: list, method: str = "relative", P: int = None):
        # method = "relative", "fitness_rank", "diversity_and_fitness_rank"
        # P only for rank methods

        if not(method == "relative" or method == "fitness_rank" or method == "diversity_and_fitness_rank"):
            raise ValueError(f'"{method}" is not a built-in probability method. Options: "relative", "fitness_rank", "diversity_and_fitness_rank", "double_fitness_diversity_rank"')

        Chromosome.calculate_diversity(chromosomes)
        if method == "relative":
            fitness_sum = 0
            for chromosome in chromosomes:
                fitness_sum += chromosome.fitness

            for chromosome in chromosomes:
                chromosome.probability = chromosome.fitness / fitness_sum
            return
        
        if method.endswith("fitness_rank"):
            chromosomes.sort(reverse=True)
            for i in range(len(chromosomes)):
                chromosomes[i].rank = i

        if method == "diversity_and_fitness_rank":
            chromosomes.sort(key=lambda chr: chr.diversity, reverse=True)
            for i in range(len(chromosomes)):
                chromosomes[i].rank += i

        
        
        if method.endswith("rank") and P != None:
            chromosomes.sort(key=lambda chr: chr.rank)
            n = len(chromosomes)
            for i in range(n):
                if i < n-1:
                    chromosomes[i].probability = (1 - P)**i * P
                else:
                    chromosomes[i].probability = (1 - P)**i
        else:
            raise ValueError("Probability needed to calculate probabilities based on rank!")

    def calculate_diversity(chromosomes: list):
        dists = {}
        n = len(chromosomes)
        for i in range(n):
            for j in range(i, n):
                dists[(i,j)] = math.dist(chromosomes[i].genes, chromosomes[j].genes)
        
        for i in range(n):
            chromosomes[i].diversity = 0
            for j in range(n):
                if i == j:
                    continue
                if i < j:
                    chromosomes[i].diversity += dists[(i,j)]
                else:
                    chromosomes[i].diversity += dists[(j,i)]

In [64]:
def print_population(population: list[Chromosome]):
    for individual in population:
        print(f"{individual.genes} - fitness: {individual.fitness}, diversity: {individual.diversity}, probability: {individual.probability}")

In [65]:
import random
import matplotlib.pyplot as plt
import numpy as np
import copy

class GeneticAlgorithm:
    def __init__(self, fitness_function: str = "Booth", probability_method: str = "relative", selection_method: str = "roulette", mutation_type: str = "any",
                 A: float = None, P: float = None, contestants: int = 2, min_gene_value: float = -10, max_gene_value: float = 10, crossover_points: int = 1,
                 elite_count: int = 1, elite_pool: int = 0, elite_mutation_rate: float = 0.2,
                 mutation_rate: float = 0.4, mutation_amount: float = 1.5, mutation_decay: float = 1):

        if fitness_function == "Rastgirin" and A == None:
            raise ValueError("'A' value needed for Rastrigin function!")
        
        if mutation_type != "one" and mutation_type != "all" and mutation_type != "any":
            raise ValueError(f'"{mutation_type}" is not a built-in mutation type. Options: "one" (changes one gene of the chromosome), "all" (changes all genes of the chromosome), "any" (all genes are evaluated separately whether they should be mutated or not.)')
        
        if probability_method.endswith("rank") and P == None:
            raise ValueError("'P' probability parameter needed in rank-based probability calculation!")
        
        if crossover_points < 1:
            raise ValueError("At least one cross-over point in needed!")
        
        if (P != None and (P < 0 or P > 1)) or (elite_mutation_rate < 0 or elite_mutation_rate > 1) or (mutation_rate < 0 or mutation_rate > 1):
            raise ValueError("Probabilities have to be given from the [0, 1] interval!")

        if mutation_decay < 0 or mutation_decay > 1:
            raise ValueError("Mutation decay is supposed to be a value between 0 and 1!")

        if selection_method == "tournament" and contestants < 1:
            raise ValueError("At least 2 contestants needed for the tournament selection!")

        if elite_count < 0:
            elite_count = 0
        
        if elite_pool < 2:
            elite_pool = 0

        if mutation_amount < 0:
            mutation_amount *= -1
        
        random.seed()
        
        self.fitness_funtion = fitness_function
        self.probability_method = probability_method
        self.selection_method = selection_method
        self.mutation_type = mutation_type

        self.A = A
        self.P = P
        self.contestants = contestants
        self.min_gene_value = min_gene_value
        self.max_gene_value = max_gene_value
        self.crossover_points = crossover_points

        self.elite_count = elite_count
        self.elite_pool = elite_pool
        self.elite_mutation_rate = elite_mutation_rate
        self.mutation_rate = mutation_rate
        self.mutation_amount = mutation_amount
        self.mutation_decay = mutation_decay

        if (mutation_decay < 1):
            self.min_mutation = mutation_rate * mutation_decay
            self.interval = mutation_rate - self.min_mutation
            self.min_elite_mutation = elite_mutation_rate * mutation_decay
            self.elite_interval = elite_mutation_rate - self.min_elite_mutation

        self.function_values = []
        self.stop = False


    def random_chromosome(self, gene_count: int = 2) -> Chromosome:
        if gene_count < 2:
            raise ValueError("At least two genes are needed!")
        if self.fitness_funtion != "Rastgirin" and gene_count > 2:
            raise ValueError(f"In the case of the {self.fitness_funtion} function, chromosomes need to have exactly two genes!")
        new_chromosome = Chromosome()
        new_chromosome.genes = randfloat(self.min_gene_value, self.max_gene_value, gene_count)
        new_chromosome.calculate_fitness(self.fitness_funtion, self.A)
        return new_chromosome
    
    def copy_chromosome(chromosome: Chromosome) -> Chromosome:
        new_chromosome = copy.deepcopy(chromosome)
        return new_chromosome
    
    #-------------------------------------------------------------------------------------------------------------
    
    def mutate_gene(gene_value: float, mutation_amount: float, min_value: float, max_value: float):
        return randfloat(max(gene_value - mutation_amount, min_value), min(gene_value + mutation_amount, max_value))

    def mutate(self, individual: Chromosome, individual_type: str = "non-elite"):
        if individual_type == "elite":
            mutation_rate = self.elite_mutation_rate
        elif individual_type == "non-elite":
            mutation_rate = self.mutation_rate
        else:
            raise ValueError(f'"{individual_type}" is not a valid chromosome type for mutation. Options: "elite", "non-elite"')

        match self.mutation_type:
            case "any":
                mutated_individual = GeneticAlgorithm.copy_chromosome(individual)
                n = len(mutated_individual.genes)
                for i in range(n):
                    should_mutate = True if random.random() < mutation_rate else False
                    if should_mutate:
                        mutated_individual.genes[i] = GeneticAlgorithm.mutate_gene(mutated_individual.genes[i], self.mutation_amount, self.min_gene_value, self.max_gene_value)
                    mutated_individual.calculate_fitness(self.fitness_funtion, self.A)
                return mutated_individual
            case "all":
                new_individual = GeneticAlgorithm.copy_chromosome(individual)
                should_mutate = True if random.random() < mutation_rate else False
                if should_mutate:
                    n = len(new_individual.genes)
                    for i in range(n):
                        gene_value = new_individual.genes[i]
                        new_individual.genes[i] = GeneticAlgorithm.mutate_gene(gene_value, self.mutation_amount, self.min_gene_value, self.max_gene_value)
                    new_individual.calculate_fitness(self.fitness_funtion, self.A)
                return new_individual
            case "one":
                new_individual = GeneticAlgorithm.copy_chromosome(individual)
                should_mutate = True if random.random() < mutation_rate else False
                if should_mutate:
                    gene_index = random.randint(0, len(new_individual.genes) - 1)
                    gene_value = new_individual.genes[gene_index]
                    new_individual.genes[gene_index] = GeneticAlgorithm.mutate_gene(gene_value, self.mutation_amount, self.min_gene_value, self.max_gene_value)
                    new_individual.calculate_fitness(self.fitness_funtion, self.A)
                return new_individual
            case _:
                raise ValueError(f'"{self.mutation_type}" is not a built-in mutation type! Options: "one", "all", "any"')

    def decay_mutation(self, generation_index, generation_count):
        x = generation_index / generation_count
        value = math.sqrt(1 - x**5)
        return value * self.interval + self.min_mutation, value * self.elite_interval + self.min_elite_mutation
    
    #--------------------------------------------------------------------------------------------------------------
    
    def max_probability(pool: list[Chromosome]) -> float:
        sum_probability = 0
        for individual in pool:
            sum_probability += individual.probability
        return sum_probability

    def select_one(pool: list[Chromosome], selection_method: str, max_probability: float = 1, contestants: int = 2):
        match selection_method:
            case "roulette":
                pool.sort(key = lambda p: p.probability)
                probability_threshold = pool[0].probability
                i = 0
                select_point = random.random() * max_probability
                while select_point > probability_threshold and i < len(pool) - 1:
                    i += 1
                    probability_threshold += pool[i].probability
                return pool[i]
            case "tournament":
                if (len(pool) - 1 < contestants): raise ValueError("At least one individual has to stay out of the tournament!")
                indices = random.sample(range(len(pool)), contestants)
                contestant_list = [pool[i] for i in indices]
                return max(contestant_list)
            case _:
                raise ValueError(f'"{selection_method}" is not a built-in selection algorithm! Options: "roulette", "tournament", "SUS"')

    def selection(pool: list[Chromosome], selection_method: str, max_probability: float = 1, contestants: int = 2):
        if selection_method == "SUS":
            pool.sort(key = lambda p: p.probability)

            probability_threshold = pool[0].probability
            i = 0
            select_point = randfloat(0, 0.5 * max_probability)
            while select_point > probability_threshold and i < len(pool) - 1:
                i += 1
                probability_threshold += pool[i].probability
            parent1 = pool[i]
            select_point += 0.5 * max_probability
            while select_point > probability_threshold and i < len(pool) - 1:
                i += 1
                probability_threshold += pool[i].probability
            parent2 = pool[i]
        else:
            parent1 = GeneticAlgorithm.select_one(pool, selection_method, max_probability, contestants)
            parent2 = GeneticAlgorithm.select_one(pool, selection_method, max_probability, contestants)
            i = 0
            while parent1 == parent2 and i < 1000:
                parent2 = GeneticAlgorithm.select_one(pool, selection_method, max_probability, contestants)
                i += 1
        return parent1, parent2

    #--------------------------------------------------------------------------------------------------------------

    def crossover(self, parent1: Chromosome, parent2: Chromosome):
        if len(parent1.genes) != len(parent2.genes):
            raise ValueError("The parents need to have the same number of genes!")
        
        child = GeneticAlgorithm.copy_chromosome(parent1)
        chromosome_length = len(child.genes)

        crossover_indices = [i+1 for i in random.sample(list(range(chromosome_length - 1)), self.crossover_points)]
        crossover_indices.sort()
        cross = False
        gene_index = 0
        ci_index = 0
        
        while gene_index < chromosome_length:
            if ci_index != len(crossover_indices) and gene_index == crossover_indices[ci_index]:
                cross = not cross
                ci_index += 1
            if cross:
                child.genes[gene_index] = parent2.genes[gene_index]
            gene_index += 1
        child.calculate_fitness(self.fitness_funtion, self.A)
        child = self.mutate(child, "non-elite")
        return child

    def next_generation(self, population: list[Chromosome]) -> list[Chromosome]:
        if self.elite_count > len(population) or self.elite_pool > len(population):
            raise ValueError("Elite individuals can't be more than the population in a generation!")
        
        if self.crossover_points > len(population[0].genes) - 1:
            raise ValueError(f"Given the number of genes, a maximum of {len(population[0].genes) - 1} crossover points are possible!")
        
        if population[0].probability == None:
            raise ValueError(f"Probabilities have to be calculated before creating the next generation!")

        
        new_population = []
        population.sort(reverse = True)

        for i in range(self.elite_count):
            new_population.append(self.mutate(population[i], "elite"))

        max_probability = 1
        if self.elite_pool > 1:
            pool = population[:self.elite_pool]
            max_probability = GeneticAlgorithm.max_probability(pool)
        else:
            pool = population
            
        if GeneticAlgorithm.all_identical_population(pool):
            self.stop = True
            return population
        
        while len(new_population) < len(population):
            parent1, parent2 = GeneticAlgorithm.selection(pool, self.selection_method, max_probability, self.contestants)
            child = self.crossover(parent1, parent2)
            new_population.append(child)
            if len(new_population) < len(population):
                new_population.append(self.mutate(parent1) if parent1 > parent2 else self.mutate(parent2))

        Chromosome.calculate_probabilities(new_population, self.probability_method, self.P)
        return new_population
    
    def run_algorithm(self, initial_population: list[Chromosome], generation_count: int = 50, diversity_limit: float = 0, visualize: bool = False, save_convergence: bool = False):        
        if visualize:
            if len(initial_population[0].genes) > 2:
                raise ValueError("Visualization only available for two dimensions!s")
            self.visualize_population(initial_population)
            plt.gca().set_title("Starting population")
       
        if save_convergence:
            best_function_values = [min(initial_population).function_value]
            worst_function_values = [max(initial_population).function_value]

        Chromosome.calculate_probabilities(initial_population, self.probability_method, self.P)
        population = copy.deepcopy(initial_population)

        for i in range(generation_count):
            population = self.next_generation(population)
            if self.stop:
                print(f"Stopped after {i+1} generations")
                break

            if i == generation_count // 2 and visualize:
                self.visualize_population(population)
                plt.gca().set_title("Middle generation")
            if save_convergence:
                best_function_values.append(min(population).function_value)
                worst_function_values.append(max(population).function_value)
            if diversity_limit > 0 and max(population, key=lambda i: i.diversity).diversity < diversity_limit:
                break
            if self.mutation_decay < 1:
                self.mutation_rate, self.elite_mutation_rate = self.decay_mutation(i, generation_count)

        if visualize:
                self.visualize_population(population)
                plt.gca().set_title(f"Last generation ({i+1})")

        if save_convergence:
            return population, best_function_values, worst_function_values
        return population

    #--------------------------------------------------------------------------------------------------------------
    def all_identical_population(population: list[Chromosome]) -> bool:
        first = population[0]
        for individual in population:
            if individual != first:
                return False
        return True

    def visualize_population(self, population: list[Chromosome]):
        if self.function_values == []:
            XY = np.linspace(self.min_gene_value, self.max_gene_value, 200)
            for y in reversed(XY):
                z = []
                for x in XY:
                    z.append(fitness_functions(self.fitness_funtion, self.A, [x,y]))
                self.function_values.append(z)
            del z, XY
        
        _, ax = plt.subplots()
        ax.imshow(self.function_values)
        for individual in population:
            x, y = individual.genes
            new_range = len(self.function_values)
            x = new_range*(x - self.min_gene_value)/(self.max_gene_value - self.min_gene_value)
            y = new_range*(self.max_gene_value - y)/(self.max_gene_value - self.min_gene_value)
            plt.plot(x, y, 'r*')

        n_ticks = 5
        ticks = np.linspace(0, len(self.function_values), n_ticks)
        ticklabels = np.round(np.linspace(self.min_gene_value, self.max_gene_value, n_ticks), 2)
        plt.xticks(ticks, ticklabels)
        plt.yticks(ticks, reversed(ticklabels))
        

## Running different algorithms

#### GA_base
Parameters:
- function calculated: Booth
- probability: relative
- selection method: roulette
- elitek száma: 1 (5 kromoszóma esetén), 2 (10 kromoszóma), 5 (20 kromoszóma), 10 (50 kromoszóma), 15 (100 kromoszóma)

In [ ]:
GA_base = GeneticAlgorithm(elite_count=10)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_base.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_base.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_base.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(len(bfv))
print(best.function_value)
best.genes

In [ ]:
plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

### Booth

#### GA_BRT
Parameters:
- function calculated: Booth
- probability: relative
- selection method: tournament

In [54]:
GA_BRT = GeneticAlgorithm(elite_count=15, selection_method="tournament", contestants=5, elite_pool=80, elite_mutation_rate = 0.2, mutation_rate=0.4, mutation_decay=0.7, mutation_type="any")

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BRT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BRT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BRT.run_algorithm(population, generation_count=20, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_BRS
Parameters:
- function calculated: Booth
- probability: relative
- selection method: Stochastic Universal Sampling

In [185]:
GA_BRS = GeneticAlgorithm(selection_method="SUS", elite_count=15, elite_pool=80, mutation_decay=0.5, elite_mutation_rate=0.3,mutation_amount=0.5, mutation_type="one")

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BRS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BRS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BRS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_BFR
Parameters:
- function calculated: Booth
- probability: fitness rank
- selection method: roulette

In [1697]:
GA_BfR = GeneticAlgorithm(probability_method="fitness_rank", P=0.35, elite_count=20, elite_pool=80,mutation_decay=0.8, mutation_amount=0.5)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BfR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BfR.run_algorithm(population, generation_count=10000, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BfR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

last_few = 80
#plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.fill_between(range(last_few), bfv[-last_few:], wfv[-last_few:], color="lightgray")
plt.plot(bfv[-last_few:], linewidth=2)
plt.plot(wfv[-last_few:], linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_BFT
Parameters:
- function calculated: Booth
- probability: fitness rank
- selection method: tournament

In [1712]:
GA_BfT = GeneticAlgorithm(probability_method="fitness_rank", selection_method="tournament", contestants=3, P=0.35, elite_pool=80, elite_count=15, mutation_amount=0.5, mutation_rate=0.7, mutation_type="any", elite_mutation_rate=0.5, mutation_decay=0.5)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BfT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BfT.run_algorithm(population, generation_count=10000, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BfT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_BFS
Parameters:
- function calculated: Booth
- probability: fitness rank
- selection method: SUS

In [308]:
GA_BfS = GeneticAlgorithm(probability_method="fitness_rank", selection_method="SUS", P=0.35, elite_count=15, elite_pool=80, mutation_amount=1, mutation_decay=0.7, mutation_type="one")

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BfS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BfS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BfS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_BDR
Parameters:
- function calculated: Booth
- probability: diversity and fitness rank
- selection method: roulette

In [386]:
GA_BDR = GeneticAlgorithm(probability_method="diversity_and_fitness_rank", P=0.35, elite_count=15, elite_pool=50, mutation_amount=1, mutation_rate=0.6, elite_mutation_rate=0.4, mutation_decay=0.7, mutation_type="one")

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BDR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BDR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BDR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_BDT
Parameters:
- function calculated: Booth
- probability: diversity and fitness rank
- selection method: tournament

In [491]:
GA_BDT = GeneticAlgorithm(probability_method="diversity_and_fitness_rank", selection_method="tournament", contestants=3, P=0.3, elite_count=15, elite_pool=80, mutation_decay=0.6, mutation_rate=0.6, elite_mutation_rate=0.4, mutation_amount=1, mutation_type="any")

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BDT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BDT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BDT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_BDS
Parameters:
- function calculated: Booth
- probability: diversity and fitness rank
- selection method: SUS

In [562]:
GA_BDS = GeneticAlgorithm(probability_method="diversity_and_fitness_rank", selection_method="SUS", P=0.2, elite_count=15, elite_pool=60, mutation_amount=1, mutation_type="one")

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_BDS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_BDS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_BDS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

### Lévi

#### GA_LRR

In [1824]:
GA_LRR = GeneticAlgorithm(fitness_function="Lévi", elite_count=15, elite_pool=80, mutation_amount=1)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LRR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LRR.run_algorithm(population, generation_count=1000, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LRR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

first_few=100
plt.fill_between(range(first_few), bfv[:first_few], wfv[:first_few], color="lightgray")
plt.plot(bfv[:first_few], linewidth=2)
plt.plot(wfv[:first_few], linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LRT

In [1837]:
GA_LRT = GeneticAlgorithm(fitness_function="Lévi", selection_method="tournament", contestants=3, elite_count=15, elite_pool=80, mutation_type="one",mutation_amount=1)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LRT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LRT.run_algorithm(population, generation_count=1000, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LRT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

first_few =100
plt.fill_between(range(first_few), bfv[:first_few], wfv[:first_few], color="lightgray")
plt.plot(bfv[:first_few], linewidth=2)
plt.plot(wfv[:first_few], linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LRS

In [1844]:
GA_LRS = GeneticAlgorithm(fitness_function="Lévi", selection_method="SUS", elite_count=15, elite_pool=80, mutation_amount=1, mutation_decay=0.75)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LRS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LRS.run_algorithm(population, generation_count=1000, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LRS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

first_few = 100
plt.fill_between(range(first_few), bfv[:first_few], wfv[:first_few], color="lightgray")
plt.plot(bfv[:first_few], linewidth=2)
plt.plot(wfv[:first_few], linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LFR

In [771]:
GA_LFR = GeneticAlgorithm(fitness_function="Lévi", probability_method="fitness_rank", P=0.3, elite_count=15, elite_pool=80, mutation_amount=1)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LFR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LFR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LFR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LFT

In [848]:
GA_LFT = GeneticAlgorithm(fitness_function="Lévi", probability_method="fitness_rank", P=0.3, selection_method="tournament", contestants=5, elite_count=15, elite_pool=80, mutation_amount=0.5)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LFT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LFT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LFT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LFS

In [887]:
GA_LFS = GeneticAlgorithm(fitness_function="Lévi", probability_method="fitness_rank", P=0.35, selection_method="SUS", elite_count=15, elite_pool=80, mutation_rate=0.5, elite_mutation_rate=0.3, mutation_decay=0.75)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LFS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LFS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LFS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LDR

In [931]:
GA_LDR = GeneticAlgorithm(fitness_function="Lévi", probability_method="diversity_and_fitness_rank", P=0.35, elite_count=15, elite_pool=50, mutation_decay=0.75, mutation_amount=1, mutation_type="one")

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LDR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LDR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LDR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LDT

In [1787]:
GA_LDT = GeneticAlgorithm(fitness_function="Lévi", probability_method="diversity_and_fitness_rank", P=0.2, selection_method="tournament", contestants=5, elite_count=15, elite_pool=65, mutation_amount=1.5, mutation_decay=0.5)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LDT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LDT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LDT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_LDS

In [1818]:
GA_LDS = GeneticAlgorithm(fitness_function="Lévi", probability_method="diversity_and_fitness_rank", P=0.15, selection_method="SUS", elite_count=15, elite_pool=80, mutation_amount=0.5)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_LDS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_LDS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_LDS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

### Rastgirin

#### GA_R2RR

In [1174]:
GA_R2RR = GeneticAlgorithm(fitness_function="Rastgirin", mutation_type="any", mutation_amount=1, mutation_rate=0.5, elite_mutation_rate=0.4, mutation_decay=0.65, elite_count=15, elite_pool=80, A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2RR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2RR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2RR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2RT

In [1225]:
GA_R2RT = GeneticAlgorithm(fitness_function="Rastgirin", mutation_rate=0.6, elite_mutation_rate=0.2, mutation_decay=0.3, mutation_amount=1.5, elite_count=15, elite_pool=80, contestants=3, selection_method="tournament", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2RT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2RT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2RT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2RS

In [1730]:
GA_R2RS = GeneticAlgorithm(fitness_function="Rastgirin", elite_count=15, elite_pool=50, mutation_rate=0.5, elite_mutation_rate=0.4,mutation_amount=1, selection_method="SUS", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2RS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2RS.run_algorithm(population, generation_count=10000, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2RS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2FR

In [1334]:
GA_R2FR = GeneticAlgorithm(fitness_function="Rastgirin", elite_count=15, elite_pool=80, P=0.2, mutation_type="one", mutation_decay=0.6, mutation_amount=2, probability_method="fitness_rank", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2FR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2FR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2FR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2FT

In [1745]:
GA_R2FT = GeneticAlgorithm(fitness_function="Rastgirin", elite_count=15, elite_pool=70, P=0.2, contestants=3, selection_method="tournament", probability_method="fitness_rank", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2FT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2FT.run_algorithm(population, generation_count=10000, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2FT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2FS

In [1418]:
GA_R2FS = GeneticAlgorithm(fitness_function="Rastgirin", elite_count=15, elite_pool=80, mutation_amount=1, P=0.3, selection_method="SUS", probability_method="fitness_rank", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2FS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2FS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2FS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2DR

In [1470]:
GA_R2DR = GeneticAlgorithm(fitness_function="Rastgirin", elite_count=20, elite_pool=50, mutation_amount=1, mutation_decay=0.5, P=0.35, probability_method="diversity_and_fitness_rank", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2DR.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2DR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2DR.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2DT

In [1519]:
GA_R2DT = GeneticAlgorithm(fitness_function="Rastgirin", elite_count=15, elite_pool=50,mutation_type="one", mutation_decay=0.3, mutation_rate=0.7, elite_mutation_rate=0.5, mutation_amount=1, P=0.35, contestants=3, selection_method="tournament", probability_method="diversity_and_fitness_rank", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2DT.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2DT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2DT.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### GA_R2DS

In [1581]:
GA_R2DS = GeneticAlgorithm(fitness_function="Rastgirin", elite_count=15, elite_pool=30, mutation_amount=1, mutation_rate=0.7, mutation_type="one", elite_mutation_rate=0.3, mutation_decay=0.3, P=0.35, selection_method="SUS", probability_method="diversity_and_fitness_rank", A=10, min_gene_value=-5.12, max_gene_value=5.12)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(GA_R2DS.random_chromosome())

In [ ]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = GA_R2DS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=True, diversity_limit=0.5)
else:
    final_population = GA_R2DS.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.plot(bfv, linewidth=2)
plt.plot(wfv, linewidth=2)
plt.gca().set_title("Best and worst values per generation")

#### n-dimenziós Rastgirin

In [1678]:
n_dim_rastgirin = GeneticAlgorithm(fitness_function="Rastgirin", probability_method="fitness_rank", P=0.2, selection_method="SUS", contestants=5, crossover_points=100, mutation_rate=0.6, elite_mutation_rate=0.4, mutation_decay=0.5, elite_count=20, elite_pool=75, mutation_type="one", min_gene_value=-5.12, max_gene_value=5.12, A=10)

population: list[Chromosome] = []
population_size = 100
for i in range(population_size):
    population.append(n_dim_rastgirin.random_chromosome(gene_count=500))


In [1679]:
save_convergence = True
if save_convergence:
    final_population, bfv, wfv = n_dim_rastgirin.run_algorithm(population, generation_count=30000, save_convergence=True, diversity_limit=0.5)
else:
    final_population = n_dim_rastgirin.run_algorithm(population, generation_count=100, visualize=True, save_convergence=False)

In [ ]:
best = max(final_population)
print(best.function_value)
print(best.genes)

last_few = 10000
#plt.fill_between(range(len(bfv)), bfv, wfv, color="lightgray")
plt.fill_between(range(last_few), bfv[-last_few:], wfv[-last_few:], color="lightgray")
plt.plot(bfv[-last_few:], linewidth=2)
plt.plot(wfv[-last_few:], linewidth=2)
plt.gca().set_title("Best and worst values per generation")